## Generating a Table of Global Fluxes

This notebook reads in the CESM1 historical run (for CMIP5),
the ensemble of 11 CESM2 historical runs (for CMIP6),
and also the four SSP CESM2 ensembles (for CMIP6).
A table is generated containing values listed [issue #6](https://github.com/marbl-ecosys/cesm2-marbl/issues/6)


> * Net primary production (PgC/yr) (`photoC_TOT_zint`)
> * Diatom primary production (%)   (`photoC_diat_zint`)
> * Sinking POC at 100 m (PgC/yr)   (`POC_FLUX_100m`)
> * Sinking CaCO3 at 100 m (PgC/yr) (`CaCO3_FLUX_100m`)
> * Rain ratio (CaCO3/POC) 100 m    (ratio of two above)
> * Nitrogen fixation (TgN/yr)      (`diaz_Nfix`)
> * Nitrogen deposition (TgN/yr)    (`NOx_FLUX` + `NHy_FLUX`)
> * Denitrification (TgN/yr)        (`DENITRIF`)
> * N cycle imbalance = deposition + fixation - denitrification (TgN/yr) # deposition = N* [see Kristen's notebook -- Biological Diagnostics?]
> * Air–sea CO2 flux (PgC yr21)     (`FG_CO2`)
> * Mean ocean oxygen (uM = umol/L = mmol/m^3)    (`O2`)
> * Volume where O2 <80 mmol/m^3 (10^15 m^3) # based on others
> * Volume where O2 <60 mmol/m^3 (10^15 m^3) # based on others
> * Volume where O2 <5 mmol/m^3 (10^15 m^3)  # based on others

Values will be computed one at a time, due to an issue with `xr.merge` and trying to read multiple variables at once.

### This notebook uses several python packages

The watermark package shows the version number used to help others recreate this environment.

In [1]:
%matplotlib inline
import os

import cftime

import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean

import cartopy
import cartopy.crs as ccrs

import esmlab

import intake
import intake_esm
import ncar_jobqueue
from dask.distributed import Client
from pint import UnitRegistry
units = UnitRegistry()

%load_ext watermark
%watermark -a "Mike Levy" -d -iv -m -g -h

cmocean       2.0
pandas        0.25.3
esmlab        2019.4.27.post55
intake        0.5.3
xarray        0.14.0
intake_esm    2019.10.15.post40
ncar_jobqueue 2019.10.16.1
cftime        1.0.3.4
numpy         1.17.3
cartopy       0.17.0
matplotlib    3.1.2
Mike Levy 2019-11-20 

compiler   : GCC 7.3.0
system     : Linux
release    : 3.10.0-693.21.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 72
interpreter: 64bit
host name  : casper02
Git hash   : da5e50c405b1687e87c5bc2743bf38263d7455d3


#### Spin up a dask cluster

Some of these computations take a while

In [2]:
cluster = ncar_jobqueue.NCARCluster(project='P93300606')
client = Client(cluster)
client

Client Scheduler: tcp://128.117.181.208:41623 Dashboard: https://jupyterhub.ucar.edu/dav/user/mlevy/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
cluster.scale(8)

### Read the intake_esm datastores

The `intake_esm` package is used to help identify which files belong in each experiment.
The `get_var_from_catalogs()` function is a wrapper to read specific files.

In [4]:
catalogs = dict()
catalogs['cesm2'] = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/campaign-cesm2-cmip6-timeseries.json')

#cesm1 = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip5_NOT_CMORIZED.json')
catalogs['cesm1'] = intake.open_esm_datastore('/glade/work/mlevy/intake-esm-collection/json/glade-cesm1-cmip5-timeseries.json')

In [5]:
# NOTE: 1991-01-01 0:00:00 is the time stamp on the Dec 1990 monthly average
#       So slice("1991", "2001") would actually return Dec 1990 - Nov 2000
#       Specifying a day mid-month gets us to Jan 1991 - Dec 2000
#       (this can be verified by looking at time bounds)
time_slices_hist = slice("1990-01-15", "2000-01-15")
time_slices_SSP = slice("2090-01-15", "2100-01-15")
time_slices = dict()

time_slices['cesm1_PI'] = slice(989*12, 999*12) # cfunits doesn't years too far in past; this is 990-01-15 - 1000-01-15
time_slices['cesm1_hist'] = time_slices_hist
time_slices['cesm2_hist'] = time_slices_hist
time_slices['cesm2_SSP1-2.6'] = time_slices_SSP
time_slices['cesm2_SSP2-4.5'] = time_slices_SSP
time_slices['cesm2_SSP3-7.0'] = time_slices_SSP
time_slices['cesm2_SSP5-8.5'] = time_slices_SSP

def get_var_from_catalogs(catalogs, variable):
    datasets = dict()
    
    dq = dict()
    # CESM1 is historical only, CESM2 also has SSPs
    # Note some variable rename shenanigans to account for changes between CESM1 and CESM2
    if variable == 'CaCO3_FLUX_100m':
        cesm1_var = 'CaCO3_FLUX_IN'
    elif variable == 'POC_FLUX_100m':
        cesm1_var = 'POC_FLUX_IN'
    else:
        cesm1_var = variable
    dq['cesm1'] = catalogs['cesm1'].search(experiment=['historical', 'piControl'], variable=cesm1_var).to_dataset_dict(cdf_kwargs={'chunks':{'time': 48}})
    dq['cesm2'] = catalogs['cesm2'].search(experiment=['historical', 'SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5'], variable=variable).to_dataset_dict(cdf_kwargs={'chunks':{'time': 48}})

    # Define datasets
    datasets['cesm1_PI'] = dq['cesm1']['ocn.piControl.pop.h']
    datasets['cesm1_hist'] = dq['cesm1']['ocn.historical.pop.h']
    datasets['cesm2_hist'] = dq['cesm2']['ocn.historical.pop.h']
    # UNCOMMENT LINES BELOW WHEN HAPPY WITH FULL TABLE
#     datasets['cesm2_SSP1-2.6'] = dq['cesm2']['ocn.SSP1-2.6.pop.h']
#     datasets['cesm2_SSP2-4.5'] = dq['cesm2']['ocn.SSP2-4.5.pop.h']
#     datasets['cesm2_SSP3-7.0'] = dq['cesm2']['ocn.SSP3-7.0.pop.h']
    datasets['cesm2_SSP5-8.5'] = dq['cesm2']['ocn.SSP5-8.5.pop.h']

    keep_vars = ['z_t', 'z_t_150m', 'dz', 'TAREA', 'TLONG', 'TLAT', 'time', 'time_bound', 'member_id', 'ctrl_member_id'] + [variable, cesm1_var]
    for exp in datasets:
        if exp == 'cesm1_PI':
            # Need isel instead of sel since PI slice is in index space rather than years
            datasets[exp] = datasets[exp].drop([v for v in datasets[exp].variables if v not in keep_vars]).isel(time=time_slices[exp])
        else:
            datasets[exp] = datasets[exp].drop([v for v in datasets[exp].variables if v not in keep_vars]).sel(time=time_slices[exp])
        if variable not in datasets[exp]:
            datasets[exp] = datasets[exp].rename({cesm1_var : variable}).isel(z_t=10)  # 100m is top of 11th level, or z_t = 10 counting from 0
        if (cesm1_var != variable) and (cesm1_var in datasets[exp]):
            datasets[exp].drop(cesm1_var)

    return(datasets)


In [6]:
# Set up units
integral_units = dict()
integral_units['area'] = dict()
integral_units['volume'] = dict()

# Read in any 3D variable to get dataset containing TAREA and z_t
# And set up a DataArray to compute total volume
tmp_var = 'DENITRIF'
tmp_data = get_var_from_catalogs(catalogs, tmp_var)

for exp in tmp_data:
    integral_units['area'][exp] = units[tmp_data[exp]['TAREA'].attrs['units']]
    integral_units['volume'][exp] = integral_units['area'][exp] * units[tmp_data[exp]['dz'].attrs['units']]

# Compute total volume of ocean
# Sum wgt over active ocean cells
vol_exp = exp # hold onto experiment used for computing volume
tmp_data2 = tmp_data[exp][tmp_var].isel(time=0, member_id=0).copy()
tmp_data2.data = np.logical_not(np.isnan(tmp_data[exp][tmp_var].isel(time=0, member_id=0).data))
total_volume = (tmp_data2 * tmp_data[exp]['TAREA'].isel(time=0) * tmp_data[exp]['dz'].isel(time=0)).sum().values
print(f'POP ocean volume: {(total_volume * integral_units["volume"][exp]).to("L")}')

# Estimate total volume in ocean
est_depth = 4*np.pi*0.71*((6400*units['km'])**2)*(3.7*units['km'])
print(f'Estimated ocean volume: {est_depth.to("L")}')

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 2 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)
POP ocean volume: 1.325121430696603e+21 liter
Estimated ocean volume: 1.3521656055366266e+21 liter


### Individual Table Computations

In this section, we compute each of the requested values for each dataset

#### Net primary production (PgC/yr)

CESM1 doesn't have `photoC_TOT_zint`

#### Diatom primary production (%)

CESM1 doesn't have `photoC_diat_zint`

#### Sinking POC at 100 m (PgC/yr)

CESM1 doesn't have `POC_FLUX_100m`

#### Sinking CaCO3 at 100 m (PgC/yr)

CESM1 doesn't have `CaCO3_FLUX_100m`

#### Rain ratio (CaCO3/POC) 100 m

Missing necessary vars to compute

#### Nitrogen deposition (TgN/yr)

#### Denitrification (TgN/yr)

In [8]:
all_data = dict()

# Process for updating intake-esm catalog
#       1. download all data from HPSS via get_ocn_cmip5_files.sh
#       2. rm /glade/u/home/mlevy/.intake_esm/collections/CESM1-CMIP5.nc
#       3. regenerate it via Anderson's legacy intake-esm
#       4. re-run build intake collections notebook
#       5. commit change to .csv.gz in /glade/work/mlevy/intake-esm-collection/csv.gz/

vars = ['POC_FLUX_100m', 'FG_CO2', 'CaCO3_FLUX_100m', 'O2', 'diaz_Nfix', 'NOx_FLUX', 'NHy_FLUX', 'DENITRIF']
#vars = ['NOx_FLUX', 'NHy_FLUX', 'DENITRIF']
for var in vars:
    all_data[var] = get_var_from_catalogs(catalogs, var)

# Verify time bounds for each experiment
for exp in all_data[vars[0]]:
    bounds = list(all_data[vars[0]][exp].time_bound.values[ind] for ind in [(0,0), (-1,1)])
    print(f'Experiment: {exp}\nBounds\n----\n{bounds}\n\n')

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 2 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 2 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 2 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 2 group(s)
--> The keys 

In [9]:
def compute_global_averages(datasets, integral_units, variable):
    experiments = list(datasets[variable].keys())
    wgts = datasets[variable][experiments[0]]['TAREA'].isel(time=0)
    dims = ['nlat', 'nlon']
    unit_key = 'area'
    if 'z_t_150m' in datasets[variable][experiments[0]][variable].dims:
        wgts = wgts * datasets[variable][experiments[0]]['dz'].isel(time=0, z_t=slice(0,15))
        wgts = wgts.rename({'z_t' : 'z_t_150m'})
        dims.append('z_t_150m')
        unit_key = 'volume'
    elif 'z_t' in datasets[variable][experiments[0]][variable].dims:
        wgts = wgts * datasets[variable][experiments[0]]['dz'].isel(time=0)
        dims.append('z_t')
        unit_key = 'volume'
    glb_avg = dict()
    new_units = dict()
    for exp in experiments:
        glb_avg[exp] = esmlab.weighted_sum(datasets[variable][exp][variable], dim=dims, weights=wgts).to_dataset(name=variable)
        old_units = units[datasets[variable][exp][variable].attrs['units']]
        new_units[exp] = old_units*integral_units[unit_key][exp]
        glb_avg[exp]
    return glb_avg, new_units

In [10]:
ann_avg = dict()
new_units = dict()
for variable in all_data:
    glb_avgs, new_units[variable] = compute_global_averages(all_data, integral_units, variable)
    ann_avg[variable] = dict()
    for exp in glb_avgs:
        glb_avgs[exp]['time_bound'] = all_data[variable][exp]['time_bound']
        ann_avg[variable][exp] = esmlab.resample(glb_avgs[exp], freq='ann')

## Reduce Data Sets

The following table shows global averages (also averaged over specified time slices)

In [11]:
# Define final units
final_units = dict()
final_units['DENITRIF'] = 'Tg/year'
final_units['NOx_FLUX'] = 'Tg/year'
final_units['NHy_FLUX'] = 'Tg/year'
final_units['diaz_Nfix'] = 'Tg/year'
final_units['FG_CO2'] = 'Pg/year'
final_units['O2'] = 'uM'
final_units['POC_FLUX_100m'] = 'Pg/year'
final_units['CaCO3_FLUX_100m'] = 'Pg/year'

# Define conversion factors
conversions = dict()

### TODO: RUN CONVERSIONS BY MATT
# Final units are TgN / yr
gN = 14.007 * units['g'] / units['mol']
gC = 12.01 * units['g'] / units['mol']
conversions['DENITRIF'] = gN
conversions['NOx_FLUX'] = gN
conversions['NHy_FLUX'] = gN
conversions['diaz_Nfix'] = gN
conversions['FG_CO2'] = gC
conversions['POC_FLUX_100m'] = gC
conversions['CaCO3_FLUX_100m'] = gC
conversions['O2'] = 1/(total_volume * integral_units["volume"][vol_exp]) # Want O2 to be average concentration

In [12]:
def get_time_and_ensemble_mean(variable, ann_avg, exp, new_units, conversions, final_units):
    return((ann_avg[variable][exp][variable].mean(['time', 'member_id']).values *
            new_units[variable][exp] *
            conversions[variable]
           ).to(final_units[variable]))

In [13]:
%%time

experiments = list(ann_avg[vars[0]].keys())
diagnostic_values = dict()
for exp in experiments:
    diagnostic_values[exp] = dict()
    # Compute each value by hand
    print(f'100m POC flux for {exp}')
    diagnostic_values[exp]['Sinking POC at 100 m (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('POC_FLUX_100m', ann_avg, exp, new_units, conversions, final_units)

    print(f'100m CaCO3 flux for {exp}')
    diagnostic_values[exp]['Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('CaCO3_FLUX_100m', ann_avg, exp, new_units, conversions, final_units)

    print(f'100m rain rate for {exp}')
    diagnostic_values[exp]['Rain ratio (CaCO$_3$/POC) at 100 m'] = diagnostic_values[exp]['Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$)'] / \
                                                                   diagnostic_values[exp]['Sinking POC at 100 m (PgC yr$^{-1}$)']
    
    print(f'Oxygen for {exp}')
    diagnostic_values[exp]['Mean ocean oxygen ($\mu$M)'] = get_time_and_ensemble_mean('O2', ann_avg, exp, new_units, conversions, final_units)
        
    print(f'Air-Sea CO2 Flux for {exp}')
    diagnostic_values[exp]['Air–sea CO2 flux (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('FG_CO2', ann_avg, exp, new_units, conversions, final_units)
        
    print(f'Computing Nfixation for {exp}')
    diagnostic_values[exp]['Nitrogen fixation (TgN yr$^{-1}$)'] = get_time_and_ensemble_mean('diaz_Nfix', ann_avg, exp, new_units, conversions, final_units)
    
    print(f'Computing Ndep for {exp}')
    diagnostic_values[exp]['Nitrogen deposition (TgN yr$^{-1}$)'] = get_time_and_ensemble_mean('NOx_FLUX', ann_avg, exp, new_units, conversions, final_units) + \
                                                                    get_time_and_ensemble_mean('NHy_FLUX', ann_avg, exp, new_units, conversions, final_units)
    print(f'Computing Denitrif for {exp}')
    diagnostic_values[exp]['Denitrification (TgN yr$^{-1}$)'] = get_time_and_ensemble_mean('DENITRIF', ann_avg, exp, new_units, conversions, final_units)
    
    print(f'Computing Nitrogen Cycle imbalance for {exp}')
    diagnostic_values[exp]['N cycle imbalance* (TgN yr$^{-1}$)'] = diagnostic_values[exp]['Nitrogen deposition (TgN yr$^{-1}$)'] + \
                                                                   diagnostic_values[exp]['Nitrogen fixation (TgN yr$^{-1}$)'] - \
                                                                   diagnostic_values[exp]['Denitrification (TgN yr$^{-1}$)']

    if exp != experiments[-1]:
        print('\n----\n')


100m POC flux for cesm1_PI
100m CaCO3 flux for cesm1_PI
100m rain rate for cesm1_PI
Oxygen for cesm1_PI
Air-Sea CO2 Flux for cesm1_PI
Computing Nfixation for cesm1_PI
Computing Ndep for cesm1_PI
Computing Denitrif for cesm1_PI
Computing Nitrogen Cycle imbalance for cesm1_PI

----

100m POC flux for cesm1_hist
100m CaCO3 flux for cesm1_hist
100m rain rate for cesm1_hist
Oxygen for cesm1_hist
Air-Sea CO2 Flux for cesm1_hist
Computing Nfixation for cesm1_hist
Computing Ndep for cesm1_hist
Computing Denitrif for cesm1_hist
Computing Nitrogen Cycle imbalance for cesm1_hist

----

100m POC flux for cesm2_hist
100m CaCO3 flux for cesm2_hist
100m rain rate for cesm2_hist
Oxygen for cesm2_hist
Air-Sea CO2 Flux for cesm2_hist
Computing Nfixation for cesm2_hist
Computing Ndep for cesm2_hist
Computing Denitrif for cesm2_hist
Computing Nitrogen Cycle imbalance for cesm2_hist

----

100m POC flux for cesm2_SSP5-8.5
100m CaCO3 flux for cesm2_SSP5-8.5
100m rain rate for cesm2_SSP5-8.5
Oxygen for cesm2

In [14]:
# Fill a dict with (data, units) tuple [unit conversion comes later]
table_dict = dict()
diagnostic_columns = ['Gross primary production (PgC yr$^{-1}$)',
                      'Sinking POC at 100 m (PgC yr$^{-1}$)',
                      'Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$)',
                      'Rain ratio (CaCO$_3$/POC) at 100 m',
                      'Nitrogen fixation (TgN yr$^{-1}$)',
                      'Nitrogen deposition (TgN yr$^{-1}$)',
                      'Denitrification (TgN yr$^{-1}$)',
                      'N cycle imbalance* (TgN yr$^{-1}$)',
                      'Air–sea CO2 flux (PgC yr$^{-1}$)',
                      'Diatom primary production (%)',
                      'Mean ocean oxygen ($\mu$M)',
                      'OMZ volume (10$^{16}$ m$^3$; <20 $\mu$M)'
                     ]
experiment_longnames={'cesm1_hist' : '1990s (CESM1)',
                      'cesm1_PI' : 'preindustrial (CESM1)',
                      'cesm2_hist' : '1990s (CESM2)',
                      'cesm2_SSP5-8.5' : 'RCP85 2090s (CESM2)'}

table_dict['Diagnostic'] = []
for variable in diagnostic_columns:
    table_dict['Diagnostic'].append(variable)
    for exp in experiments:
        if experiment_longnames[exp] not in table_dict:
            table_dict[experiment_longnames[exp]] = []
        try:
            table_dict[experiment_longnames[exp]].append(diagnostic_values[exp][variable].magnitude)
        except:
            table_dict[experiment_longnames[exp]].append('-')

pd.DataFrame(table_dict)

,Diagnostic,preindustrial (CESM1),1990s (CESM1),1990s (CESM2),RCP85 2090s (CESM2)
0,Gross primary production (PgC yr$^{-1}$),-,-,-,-
1,Sinking POC at 100 m (PgC yr$^{-1}$),8.02219,8.03133,7.08422,6.74271
2,Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$),0.772988,0.755342,0.769154,0.814445
3,Rain ratio (CaCO$_3$/POC) at 100 m,0.0963562,0.0940494,0.108573,0.120789
4,Nitrogen fixation (TgN yr$^{-1}$),175.491,169.213,242.186,287.299
5,Nitrogen deposition (TgN yr$^{-1}$),6.70941,30.0559,35.8153,38.8297
6,Denitrification (TgN yr$^{-1}$),182.661,193.974,193.105,265.682
7,N cycle imbalance* (TgN yr$^{-1}$),-0.461057,5.29435,84.8963,60.4462
8,Air–sea CO2 flux (PgC yr$^{-1}$),-0.0559473,1.7635,1.8257,5.32442
9,Diatom primary production (%),-,-,-,-
